In [ ]:
!pip install matplotlib
!pip install pyppeteer

!pip install youtube-transcript-api 
!pip install -U scikit-learn scipy matplotlib
!pip install sklearn 
!pip install pandas
!pip install numpy

!pip install spacy
!python -m spacy download en_core_web_lg 
!pip install nltk


!pip install soundfile
!pip install transformers[torch]
!pip install torchaudio --user 
!pip install librosa --user 
!pip install PySoundFile --user
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git

!pip install gensim

!pip install audb
!pip install opensmile

!pip install speechbrain --user
!pip install --upgrade pip
!pip install speechbrain --user



In [ ]:

import requests, json
import subprocess, os, argparse, datetime, re, sys,math  ,time
from PIL import Image
from io import BytesIO
from collections import Counter
from string import punctuation

from dataclasses import dataclass
from typing import Optional, Tuple

import pandas as pd  
import matplotlib.pyplot as plt
import numpy as np 


from sklearn.cluster import KMeans  #For applying KMeans
from sklearn.metrics import silhouette_score , calinski_harabasz_score , davies_bouldin_score  ,pairwise_distances_argmin


import nltk
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans


import spacy

nlp = spacy.load("en_core_web_lg")

import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

import torch.nn as nn
import torch.nn.functional as F
import torchaudio



from transformers import AutoConfig, Wav2Vec2FeatureExtractor


from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

from transformers.file_utils import ModelOutput

from speechbrain.pretrained.interfaces import foreign_class


import librosa
import IPython.display as ipd

import audb
import audiofile
import opensmile

from scipy.io import wavfile


In [ ]:


def getUrlsFromIds(lst_links_str,isId = True):
    lst_links_str_cpy = lst_links_str.copy()
    if (isId) :
        for i in range(len(lst_links_str_cpy)) :
            lst_links_str_cpy[i] = "https://www.youtube.com/watch?v="+lst_links_str_cpy[i] if not( "https://www.youtube.com/watch?v=" in lst_links_str_cpy[i]) else lst_links_str_cpy[i]
    return lst_links_str_cpy


def getContentFromURL(url):
    return requests.get(url).text

def getvideoInfo(  videoId):
    url = f'https://yt.lemnoslife.com/videos?part=mostReplayed&id={videoId}'
    content = getContentFromURL(url)
    data = json.loads(content)
    return data

def getvideosInfos(videoIds , isId = True ):
    return [getvideoInfo(videoId) for videoId in videoIds]
    

def getHeatMarkersDataVideo(data):
    return data["items"][0]['mostReplayed']['heatMarkers'] if bool(data["items"][0] ) else []

def getHeatMarkersDatasVideos(data):
    return [getHeatMarkersDataVideo(videoData) for videoData in data]


In [ ]:

def getLenVideo(data_wh):

    d=data_wh[-1]['heatMarkerRenderer']
    len=d['timeRangeStartMillis' ] +d['markerDurationMillis']


In [ ]:
def filterMoreThanMean(data, fields ,x_field,y_field,z_field, mean):
    res=[]
    m=0
    l = len(data)
    for i in range(l) :
        d1=data[i]
        for field in fields : 
            d1=d1[field]
        m+=d1[z_field]
        if d1[y_field] > mean :
            res.append([d1[x_field],d1[y_field]])
            
    
    return res , m/l

def filterData(data,fields=['heatMarkerRenderer'], x_field = 'timeRangeStartMillis' , cmp_field = 'heatMarkerIntensityScoreNormalized' , z_field = 'markerDurationMillis' ,seuil=0.5):
    return filterMoreThanMean(data[2:], fields, x_field,cmp_field,z_field , seuil)


In [ ]:

def find_best_cluster (X , k_min , k_max , seed_random ):
    if(not(bool(X))):
        return [],0,[]
    
    
    fitted_kmeans = {}
    labels_kmeans = {}
    df_scores = []
    centroids= {}

    l_x = len(X)
    k_max = k_max if l_x > k_max else l_x 

    for n_clusters in range(k_min,k_max):
        
        #Perform clustering.
        kmeans = KMeans(n_clusters=n_clusters,
                        random_state=seed_random,
                        )
        labels_clusters = kmeans.fit_predict(X)
        
        #Insert fitted model and calculated cluster labels in dictionaries,
        #for further reference.
        fitted_kmeans[n_clusters] = kmeans
        labels_kmeans[n_clusters] = labels_clusters
        centroids[n_clusters]  = kmeans.cluster_centers_ 
    
        #Calculate various scores, and save them for further reference.
        silhouette = silhouette_score(X, labels_clusters)
        ch = calinski_harabasz_score(X, labels_clusters)
        db = davies_bouldin_score(X, labels_clusters)
        tmp_scores = {"n_clusters": n_clusters,
                      "silhouette_score": silhouette,
                      "calinski_harabasz_score": ch,
                      "davies_bouldin_score": -db,
                      }
        df_scores.append(tmp_scores)

    #Create a DataFrame of clustering scores, using `n_clusters` as index, for easier plotting.
    
    df_scores = pd.DataFrame(df_scores)
    df_scores.set_index("n_clusters", inplace=True)

  

    df_ = df_scores.sort_values(by=['silhouette_score','calinski_harabasz_score','davies_bouldin_score'],ascending=False)
    display(df_)

    predicted_labels = labels_kmeans[df_.index[0]]
    predicted_centroids = centroids[df_.index[0]]
    nb_cluster = df_.index[0]
    return predicted_labels ,    nb_cluster,predicted_centroids 



In [ ]:

def init_dict_dir(path_root = "data" , path_level_2 = "test" , current_path = None , level_0 = "root" , level_1 =["raw" , "meta"] , level_2 = ["txt","img","mp3"]):
    if current_path is None : 
        current_path = os.getcwd()
   
    data_path =  os.path.join(current_path,path_root)

    lst_dir_info = level_2
    dct_dir_info = {}
    for e in lst_dir_info:
        dct_dir_info.update({e:e})

    for k in dct_dir_info.keys():
        dct_dir_info[k] = os.path.join(path_level_2,dct_dir_info[k])

    lst_dir_data =  level_1
    dct_data_info = {}
    for e in lst_dir_data:
        dct_data_info.update({e:e})

    _lst = [dct_data_info]+[dct_dir_info] #level_1 + level_2 

    dct_total = {}
    for i in range(len(_lst)): # 1 : level_1_value ; 2 : level_2_value
        dct_total.update({i+1:_lst[i]})

    for k0,v in dct_total.items():
        for k in v.keys():
            v[k] = os.path.join(data_path, v[k])
            isExist = os.path.exists(v[k])
            if not isExist : 
                os.makedirs(v[k])

    dct_total.update({0:{level_0 :data_path}}) # 0: level_0_value
    return dct_total

In [ ]:
def create_link_file( ids_links_str , root_dwn_dir ,filename = "linkList.txt") :
    batch_link_dwn = os.path.join(root_dwn_dir,filename)
    lst_links_str =getUrlsFromIds( ids_links_str) 
    with open(batch_link_dwn, 'w') as f:
        for line in lst_links_str :
            f.write(f"{line}\n")
    return batch_link_dwn , lst_links_str



In [ ]:

def ytdlp_dwn( exec_ytdlp, batch_link_dwn , output_filename , isbatch = False ):  #, output_path ):
    print(exec_ytdlp, batch_link_dwn , output_filename)
    optionBatch = ["-a"] if isbatch else []
    return subprocess.call([exec_ytdlp ]+optionBatch + [ batch_link_dwn    , "--write-description" ,"--write-info-json" , "--skip-download" , "--youtube-skip-dash-manifest" , "-o" ,output_filename    ],shell=True)  #,"-P" ,output_path

In [ ]:

def index_representative_points(X,centroids ,  predicted_labels , n_cluster ):
    ret = []
    for k in range( n_cluster):
        mask = (predicted_labels == k).nonzero()[0]
        centroid = centroids[k]
        i0 = mask[pairwise_distances_argmin(centroid[None, :], X[mask])[0]]
        ret.append(i0)
    return np.array(ret)




In [ ]:
    
def getSummaryTags(tags_list) : 
        words = word_tokenize(' '.join(tags_list))
        nouns_and_verbs = []
        lemmas = []
        for n in words:
                for token in nlp(n):
                        if token.pos_ == 'NOUN' or token.pos_ == 'VERB':
                                #print(f'{token.text:{8}} {token.pos_:{6}} {token.tag_:{6}} {token.dep_:{6}} {spacy.explain(token.pos_):{20}} {spacy.explain(token.tag_)}')
                                if (token.lemma_ not in lemmas) :
                                        lemmas.append(token.lemma_)
                                        nouns_and_verbs.append(token.vector)

        if(len(nouns_and_verbs) == 0):
                return []
        
        predicted_labels , n_clusters ,centroids = find_best_cluster ( nouns_and_verbs ,2,10,1)
        print(len(centroids) )
        nouns_and_verbs = np.array(nouns_and_verbs)
        most_rpz_idx = index_representative_points(nouns_and_verbs,centroids  ,  predicted_labels , n_clusters ) 

        summary_tags = []

        for i in most_rpz_idx :
            summary_tags.append(lemmas [i])

        return summary_tags


In [ ]:

def getSubPartJson(content_file,prop_to_extract   ) :
    meta_obj_json = dict.fromkeys(prop_to_extract)
    for prop in prop_to_extract :
        meta_obj_json[prop] = content_file[prop] if( prop in content_file) else []
    return meta_obj_json


def writeData(json_object ,filename,open_type =None) :
    str_json_object = json.dumps(json_object, indent=4)
    if(open_type == None):
        open_type = 'a'
    with open(filename, open_type) as outfile:
        outfile.write(str_json_object)
        
    return filename 

def getDctDir(_id,dct_total ,pos_path,sub_dir):
    if(pos_path in dct_total and sub_dir in dct_total[pos_path]):
        meta_dwn_dir= dct_total[pos_path][sub_dir]
        meta_info_dir = os.path.join(meta_dwn_dir,_id)
        isExist = os.path.exists(meta_info_dir)
        if not isExist:
                os.makedirs(meta_info_dir)
        return meta_info_dir


def getMetaDir(_id,dct_total):
    return  getDctDir( _id, dct_total , 1 ,'meta')

def getRawDir(_id,dct_total):
    return  getDctDir( _id , dct_total , 1 ,'raw')


def getDfDescriptionFilename():
    return "fichier_test.description"

def matchArrowbaseYtParticipants( description):
    return  re.findall(r"\W*(@[^\s]+)", description )


def matchUrlParticipants (description ,reseau_video={"https://www.twitch.tv/":"twitch", "https://www.youtube.com/" :"youtube"}):
    dct_find = dict()
    for reseau in reseau_video.values():
        dct_find.update({reseau:[]})
        
    for url_reseau ,reseau in reseau_video.items() :
        url_reseau_escaped = str(url_reseau).replace("/[-[\]{}()*+?.,\\^$|#\s]/g", "\\$&")
        reg_ = url_reseau_escaped + r'[^\s]+'
        dct_find[reseau]+=re.findall(reg_, description )

    return dct_find

    

def toUrlYtParticipants(exec_ytdlp  ,participants):
    res=[]
    for p in participants[1:] :
        participant_url = "https://www.youtube.com/" + p
        if(b_verifyUrlParticipant(exec_ytdlp  , participant_url)):
            res.append(participant_url)
    return res

def getParticipants(exec_ytdlp  ,description):
    dct_res = matchUrlParticipants (description)
 
    find = matchArrowbaseYtParticipants(description)
    dct_res["youtube"]+=toUrlYtParticipants(exec_ytdlp  ,find)
    return dct_res


def retCodeCorrectRequestYtdlp():
    return 0

def b_verifyUrlParticipant(exec_ytdlp  ,url_participant):
    return subprocess.call([exec_ytdlp , "--playlist-items"  , "0"  , "--print" , "playlist:channel_url"  , url_participant],shell=True)  == retCodeCorrectRequestYtdlp()
    
def getTestDir(_id,dct_total,sub_dir):
    return  getDctDir( _id, dct_total , 2 ,sub_dir)

def writeContextMetaData(exec_ytdlp  ,data,dct_total,open_type = None) :
    _id =data['id']
    meta_info_dir = getMetaDir(data['id'],dct_total)
    meta_obj_json =getSubPartJson(data,[  "id","title","uploader","uploader_id","uploader_url","channel_id","channel_url","duration","view_count","age_limit","webpage_url","categories","tags","live_status","comment_count","like_count","channel","channel_follower_count","upload_date","availability","webpage_url_basename","webpage_url_domain","display_id","fulltitle","duration_string"])#,"language_preference"])
    print(data)
    meta_obj_json["summary_tags" ] = getSummaryTags(data["tags"])
    
    meta_raw_dir = getRawDir(_id,dct_total)
    description_filename = os.path.join(meta_raw_dir,getDfDescriptionFilename() )
    with open(description_filename, 'r') as outfile:
        description = outfile.read()
        meta_obj_json["participants"] = getParticipants(exec_ytdlp ,description)
    meta_info_filename = os.path.join(meta_info_dir,"context_meta.json")
    writeData( meta_obj_json,meta_info_filename,open_type)

def writeVideoMetaData(exec_ytdlp  , data,dct_total, nb_formats =10 ,open_type=None):
    _id =data['id']
    meta_info_dir = getMetaDir(_id,dct_total)
    meta_obj_json =getSubPartJson(data,["duration","chapters","subtitles","formats"] )
    meta_obj_json["formats"] = meta_obj_json["formats"][:nb_formats]
    meta_info_filename = os.path.join(meta_info_dir,"video_meta.json")
    writeData( meta_obj_json,meta_info_filename,open_type)
    #getDescription(data['id'])
    #writeData()




In [ ]:
def download_video ( exec_ytdlp,dct_total,output_filename , lst_ID_links_str ,root_dwn_dir):
   
    batch_link_dwn ,lst_links_str = create_link_file (lst_ID_links_str,root_dwn_dir  )
    errs_url = []
    res_url = []
    #Download des videos
    for i,IDlink in enumerate(lst_ID_links_str) :
        id_dir = getRawDir(IDlink,dct_total)
        isExist = os.path.exists(id_dir)
        if not isExist:
                os.makedirs(id_dir)
        filename = os.path.join(id_dir,output_filename )
        err_subProcess =ytdlp_dwn(exec_ytdlp , lst_links_str[i] , filename ) #batch_link_dwn : fichier contenant les liens des videos à télécharger , output_filename : fichier de sortie contenant les metadonnées des videos téléchargées
        if err_subProcess == -1 :
            errs_url.append(lst_links_str[i])
            print("Error download video" + str(lst_links_str[i]))
        else :
            res_url.append(filename+".info"+".json")
    return  res_url, errs_url

In [ ]:

def init_directories ( dct_total,exec_ytdlp,output_filename , lst_ID_links_str ,root_dwn_dir , isId = True ):

            
    outputs_full_names ,_ = download_video ( exec_ytdlp,dct_total,output_filename , lst_ID_links_str ,root_dwn_dir)

    for output_full_name in outputs_full_names :
        f = open(output_full_name)
        
        data = json.load(f)


        writeContextMetaData(exec_ytdlp,data,dct_total,open_type="w") 
        writeVideoMetaData(exec_ytdlp,data,dct_total,open_type="w")


        f.close()
  
    return outputs_full_names

In [ ]:
def getStartMs():
     return 'tStartMs'
def getDurationMs():
     return 'dDurationMs'

def getSubtitleFromListTimes( jsonSubt_events ,list_times) :
    k=0
    res = []
    sub_times = []
    b= True 
    for  time in list_times :
        lst = []

        while( b and jsonSubt_events[k][getStartMs()] + jsonSubt_events[k][getDurationMs()] < time[0] ) :
            k+=1
            if( k == len(jsonSubt_events) ) :
                b= False
        if(b):
            lst.append(jsonSubt_events[k]['segs'][0]['utf8'])
            sub_times.append([jsonSubt_events[k][getStartMs()] ,jsonSubt_events[k][getDurationMs()]  ] )
            while( b and  jsonSubt_events[k][getStartMs()] + jsonSubt_events[k][getDurationMs()] < time[1] ) :
                    k+=1
                    lst.append(jsonSubt_events[k]['segs'][0]['utf8'])
                    sub_times.append([jsonSubt_events[k][getStartMs()] ,jsonSubt_events[k][getDurationMs()]  ] )
                    if( k == len(jsonSubt_events) ) :
                        b= False

        res.append(lst)

        if(not(b)):
            return res ,sub_times
    return res ,sub_times


In [ ]:

def milliTimeto_second( time ):#attends un temps en millisecondes
    return time/1000


def rawDuration_toSeconds(duration):
    print(duration)
    return [ milliTimeto_second(e) for e in duration ]



def rawDurations_toSeconds(durations):
    return [ rawDuration_toSeconds(e ) for e  in durations ] 


def getSecondIntervalFromTimesMilli(interval_times_sort_mili) : 
    second_interval_times =[]
    for i in range(len(interval_times_sort_mili )) :
        second_interval_times.append( rawDurations_toSeconds(interval_times_sort_mili[i]))
    return second_interval_times


def secondTimeto_secminhours( time ):#attends un temps en  secondes

        seconds ,minutes= math.modf(time/60)
        heures = 0
        print(seconds, minutes)
        if minutes > 60  :
                minutes , heures= math.modf(minutes/60)
                minutes = round(minutes * 60 ,2 )
        else :
            seconds = round(seconds *0.6 ,2 )
        return [heures,minutes,seconds]

def milliTimeto_secminhours( time ):
    return secondTimeto_secminhours( milliTimeto_second( time) )
     
def milliTimeto_min( time ):
    return milliTimeto_second( time)/60

def secmin_to_str( times ):
    str_res = ""
    i=0
    print(times)
    for  time in times:
        if i == (len(times)-1):
            print("time " + str(time))
            stre = (str(time).split("."))[1][:2]
        else:
            stre = (str(time).split("."))[0] 
        print("stre " + stre)
        if( len(stre) <2 ):
            stre = "0" + stre

        str_res += ":" + stre 
        i+=1
    str_time = ["hours","minutes","seconds"]
    tail = ""
    for k in range(i,len(str_time)):
            tail  += "00:"
    
    return tail+str_res[1:]
            

def secondTimeto_str( time ):
    return secmin_to_str( secondTimeto_secminhours( time ) )
def milliTimeto_str( time ):
    return secmin_to_str( milliTimeto_secminhours( time ) )

def toSectionTimeCode( lst_times ):
    res = []
    for e in lst_times:
        res.append("-".join(e))
    return res


In [ ]:
def mergeCategories_retInterval(n_clusters, predicted_kmeans, heatMarker_flt,max=90000):
        print("MERGE CATE ")
        print( n_clusters , predicted_kmeans , heatMarker_flt)

        arr_interval = [[] for i in range(n_clusters)]
        
        for i,e in enumerate(predicted_kmeans):
            arr_interval[e].append([heatMarker_flt [i][0],heatMarker_flt [i][1]])
        arr_interval.sort(key=lambda x: x[0])

        weights = []
        intervals = []
        for e in arr_interval:
            for i in range(len(e)):
                elm = e[i][0]
                if(i == 0):
                    beg = [elm,elm]
                    weight = e[i][1]
                else:
                    if(elm < beg[0]):
                        beg[0]=elm

                    if elm > beg[1]:
                        if(elm - beg[0] > max):
                            intervals.append([beg[0],beg[1]] )
                            weights.append(weight)
                            beg[0] =  beg[1] =elm
                            weight = e[i][1]
                        else:
                            print("icic")
                            beg[1] = elm
                            weight += e[i][1]
            intervals.append([beg[0],beg[1]] )
            weights.append(weight)
              
                
        '''
            beg = arr_interval[e]
            if(elm < beg[0]):
                    beg[0]=elm

            if elm > beg[1]:
                if(elm - beg[0] > max):
                        arr_interval[e].append([beg[0],beg[1]] )
                        weights[e].append(weights[e])
                        beg[0] =  beg[1] =elm
                        weights[e] = 0
                else:
                        beg[1] = elm
                        weights[e] +=heatMarker_flt [i][1]
        '''
                        
                
                
            
        return  intervals , weights

def getIdxNotSignifiantWeight(arr_interval,weights):
 
        idx_filter = []

        for i in range(len(weights)) :
            lgth = (arr_interval[i][1] -arr_interval[i][0]) * pow(10,-4)
            weights[i] = weights[i] / lgth if lgth != 0 else weights[i]
            if(weights[i] < 0.5 ):
                idx_filter.append(i) 
        return idx_filter

def filterAndSortArr( arr_interval,weights,idx_filter=[]):
        arr_weights = np.array([weights[i] for i in range(len(weights)) if i not in idx_filter])
        arr_e = np.array([arr_interval[i] for i in range(len(arr_interval)) if i not in idx_filter])


        idx_sort = [i[0] for i in sorted(enumerate(arr_e ), key=lambda x:x[1][0])]
        arr_e_sort = np.array(arr_e)[idx_sort]
        weights_sort = np.array(arr_weights)[idx_sort]

        return arr_e_sort,weights_sort

In [ ]:
#mergeCategories_retInterval(2,[1,1,0,0,0,0,0], [[5713490, 0.6071078100432646], [5715490, 1], [23455380, 0.5440194935904227], [24358410, 0.6187778955466057], [24360410, 0.5770295807850397], [27063900, 0.5238282322906579], [29168870, 0.529939425996983]])

In [ ]:

def getFirstNFormat ( jsonSubt_formats , needs_str,  num_format = 2 ) : #suppose classe par fileSize croissant
    lst_format = []
    for jf in jsonSubt_formats:
         print(jf["format"])
         if(all([x in  jf["format"] for x in needs_str]) ):
            num_format-=1
            lst_format.append(jf)
            if(num_format == 0 ):
                return lst_format

def getFirstNMp3Format ( jsonSubt_formats , num_format = 1 ) : #suppose classe par fileSize croissant
    return getFirstNFormat ( jsonSubt_formats , ["audio only"] , num_format )

In [ ]:
def getRegexCommandToDlSection(section_timecode_todl,only_audio=True):
    lst_dwnl_section_cmd = []
    for section_t_code in section_timecode_todl :
            lst_dwnl_section_cmd+= ["--download-sections" ,"*"+section_t_code ]+ ["--extract-audio" ] if only_audio else []
    return lst_dwnl_section_cmd

def dlSectionFromTimes (times_centi ,exec_ytdlp , output_path,  ytLink ,only_audio=False ,filename = None ):
    if filename is None :
        filename = "audio_section"
    print(filename , output_path)
    section_timecode_todl_1 = [list(map( secondTimeto_str , e )) for e in times_centi ]
    print(section_timecode_todl_1)
    section_timecode_todl_2 = toSectionTimeCode(section_timecode_todl_1)
    lst_dwnl_section_cmd = getRegexCommandToDlSection(section_timecode_todl_2,only_audio)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    output_file = os.path.join(output_path , filename)
    print(output_file,ytLink,exec_ytdlp)

    return subprocess.call([exec_ytdlp  ] +lst_dwnl_section_cmd + ["--downloader",  "aria2c" ,ytLink ,"-o",  output_file+"_"+'%(autonumber)s'   ],shell=True) 

      
def writeMp3FromTimes(jsonSubt_formats , _id , times_centi , exec_ytdlp, output_path,  filename = None ):
    formatMp3 = getFirstNMp3Format ( jsonSubt_formats )
    ytLink = getUrlsFromIds([_id])[0]
    return dlSectionFromTimes (times_centi ,exec_ytdlp , output_path,  ytLink ,only_audio=True ,filename =filename   )


In [ ]:



def getTimesIntervals(lst_ID_links_str,exec_ytdlp):

    heatMarkers_datas =  getvideosInfos(lst_ID_links_str)
    heatMarkers_wh=getHeatMarkersDatasVideos(heatMarkers_datas)

    intervals_times_sort = []
    heatmarks_weights_sort = []
    max_duration = 1.5 
    for i,heatMarker_wh in enumerate(heatMarkers_wh):
        if( len(heatMarker_wh) == 0 ):
            continue
        len_video=getLenVideo(heatMarker_wh)

        
        while( True  ) :
            heatMarker_flt ,  m  =filterData(heatMarker_wh)
            print("heatMarker_flt : " + str(heatMarker_flt))#(temps , poids)

            predicted_kmeans , n_clusters ,_ = find_best_cluster ( heatMarker_flt  ,2,8 ,1)

            interval_time ,heatmark_weight = mergeCategories_retInterval(n_clusters, predicted_kmeans, heatMarker_flt)

            if(milliTimeto_min(m) <= max_duration):
                break
            else :
                tme=interval_time.copy()
                for z in range(len(tme)) :
                    tme[z][1]=tme[z][1]+m 

                tme=rawDurations_toSeconds(interval_time)

                dlSectionFromTimes (tme ,exec_ytdlp , "D:\\Master\\S2\\GestionProjet\\tmp",  getUrlsFromIds([lst_ID_links_str[i]])[0] ,only_audio=True, filename = "audio_cluster" )
                break

           
        
        idx_filter = getIdxNotSignifiantWeight(interval_time,heatmark_weight) if(milliTimeto_min(m) < max_duration  ) else []

        

        interval_time_sort , heatmark_weight_sort = filterAndSortArr(  interval_time ,heatmark_weight ,idx_filter)


        intervals_times_sort.append(interval_time_sort)
        heatmarks_weights_sort.append(heatmark_weight_sort)
    print("intervals_times_sort : " + str(intervals_times_sort))
    print("heatmarks_weights_sort : " + str(heatmarks_weights_sort))

    return intervals_times_sort,heatmarks_weights_sort



In [ ]:
def getEventSubtitle(url) :
    xer = requests.get(url)
    json_obj = json.loads(xer.text)
    return json_obj["events"]

def getFirstSubtitle(video_meta_filename ,ext = ["json3","srv2","srv3"]):
    #oepn video meta file load json and extract "subtitles" fields
    f = open(video_meta_filename)
    data = json.load(f )
    ext_found = [""for i in range(len(ext))]
    url_base = data["subtitles"]["en"]
    i=0
    url= url_base[i]
    while (bool(url) and url["ext"] != ext[0]):
        try:
            idx= ext.index(url["ext"])
            ext_found[idx]=url["ext"]
        except ValueError:
            pass
        i+=1
        url =  url_base[i] if i < len(url_base) else None
    if(bool(url)):
        i=0
        return  getEventSubtitle(url["url"]) ,ext[i]
    else:
        for i in range(len(ext)):
            if(ext_found[i] != ""):
                return  ext_found[i],ext[i] 
        return None,None



In [ ]:
def get_hotwords(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN'] # 1
    doc = nlp(text.lower()) # 2
    for token in doc:
        # 3
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        # 4
        if(token.pos_ in pos_tag):
            result.append(token.text)
                
    return result # 5

def getPosImage ( time , atm_time ,cur_dur ,  u , nb_col ):
        #print(time , atm_time , u , nb_col )
        diff_time = time - ( atm_time  -cur_dur) 
        num = (diff_time * u)/cur_dur
        num_row = math.floor(num / nb_col)
        num_col = math.floor(num % nb_col)
        return [num_row,num_col]


def getFirstNImageFormat ( jsonSubt_formats , num_format = 3 ) : #suppose classe par fileSize croissant
    return getFirstNFormat ( jsonSubt_formats , ["(storyboard)"] , num_format ) 

def getUrlImageTilesFromTimes( format  ,list_times  ) :
    k=0
    res = []
    b= True 
    atm_time = 0
    fragment = format["fragments"]
    nb_row = format["rows"]
    nb_col = format["columns"]
    u = nb_row * nb_col 

    for  z, time in enumerate(list_times) :
        lst = []

        while( b and atm_time  < time[0] ) :
            atm_time += fragment[k]["duration"]
            k+=1
            if( k == len(fragment) ) :
                b= False 
        if(b):
            pair_pos = getPosImage (time[0] ,  atm_time ,fragment[k]["duration"] , u , nb_col)
            lst.append( [fragment[k]["url"]] +[z] +  [pair_pos] )

            while( b and   atm_time  < time[1] ) :
                    k+=1
                 
                    
                    if( k == len(fragment) ) :
                        b= False
                    else:
                        atm_time += fragment[k]["duration"]
                        pair_pos = getPosImage (time[1] ,  atm_time,fragment[k]["duration"] , u , nb_col)
                        lst.append( [fragment[k]["url"]] +[z] +[pair_pos] )
                    

        res+=lst

        if(not(b)):
            return res
    return res 


def writeImageFromTiles(lst_tiles ,times ,  img_out_dir , width , height ):
     pos_url = 0 
     category_time =  1 
     pos_tiles = 2
     print(img_out_dir)
     idxs = [0 for i in range(len(times))]
     for i, tiles in enumerate(lst_tiles) :
        image_url = tiles[pos_url]
        tiles_pos = tiles[pos_tiles]
        time = times[tiles[category_time]]
        idxs[tiles[category_time]]+=1
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))
        crop_rectangle = (tiles_pos[1] * width, tiles_pos[0] * height, (tiles_pos[1] + 1) * width, (tiles_pos[0] + 1) * height)
        cropped_im = img.crop(crop_rectangle)

        filename = '_'.join(map(str,time+[idxs[tiles[category_time]]]) )+'.png'
        cropped_im.save(os.path.join(img_out_dir ,filename))
        #imgplot = plt.imshow(img)
        #plt.show()



def getFormats():
    return 'formats'


def writeImageFromTimes(jsonSubt_formats ,times_centi ,  img_out_dir  ):
    formats = getFirstNImageFormat ( jsonSubt_formats  )#recupere le dernier format : celui qui contient le plus d'images non groupé : niveau de grain le plus faible
    format = formats[len(formats)-1] 

    lst_urlsAndPos_image = getUrlImageTilesFromTimes( format  , times_centi ) #extrait du format les urls des images et les positions des images dans les tiles

    height = jsonSubt_formats[2]["height"]
    width = jsonSubt_formats[2]["width"]

    return writeImageFromTiles(lst_urlsAndPos_image ,times_centi  ,  img_out_dir , width , height )



In [ ]:

current_path = os.getcwd()

output_path = os.path.join(current_path,"test")

dct_total = init_dict_dir() 
print(dct_total)


#Path vers le git de yt-dlp
path_ytdlp = os.path.join(current_path, "yt-dlp")
exec_ytdlp = os.path.join(path_ytdlp, "yt-dlp")

#input filename info
root_dwn_dir= dct_total[0]['root']

#Output filename info 
raw_dwn_dir = dct_total[1]['raw']
output_filename = "fichier_test"




lst_ID_links_str = ["AOKBuz9yJJ8"]
outputs = init_directories ( dct_total,exec_ytdlp,output_filename , lst_ID_links_str ,root_dwn_dir )
print(lst_ID_links_str)


intervals_times_sort,heatmarks_weights_sort = getTimesIntervals(lst_ID_links_str,exec_ytdlp )

print(intervals_times_sort,heatmarks_weights_sort)



In [ ]:


print(outputs)


json_obj= {getStartMs(): 0 , getDurationMs(): 0 , "tags" : []}
for i  in range(len(intervals_times_sort )):
    _id = lst_ID_links_str[i]
    video_meta_filename = outputs[i]
    interval_time_sort = intervals_times_sort[i]
    subtitle,_ = getFirstSubtitle(video_meta_filename)
    subtitles_times ,sub_times = getSubtitleFromListTimes( subtitle ,interval_time_sort )
    k = 0
    json_to_write = []
    for z  in range(len(subtitles_times)) :
        subtitles_time= subtitles_times[z]
        for x in range(len(subtitles_time)):
            e = subtitles_time[x]
            t = sub_times[k]
            doc =  get_hotwords(e)
            json_obj_tmp = json_obj.copy()
            json_obj_tmp[getStartMs()] = t[0] 
            json_obj_tmp[getDurationMs()] = t[1] 
            json_obj_tmp["tags"] = doc
            json_to_write.append(json_obj_tmp)
            k+=1
    path_dir_txt = getTestDir(_id,dct_total,"txt") 
    filename_subtitle = os.path.join(path_dir_txt , "subtitles"+".json" )      
    with open(filename_subtitle, 'w') as outfile:
        json.dump(json_to_write, outfile)
        
    
centi_interval_times = rawDuration_toSeconds(intervals_times_sort )

for i  in range(len(intervals_times_sort  )):
    _id = lst_ID_links_str[i]
    meta_dir = getMetaDir(_id,dct_total)
    filename_video = os.path.join(meta_dir , "video_meta" +".json")
    print(filename_video)
    f2 = open(filename_video, "r")
    json_meta_vid =  json.load(f2 )
    f2.close()
    jsonSubt_formats = json_meta_vid[getFormats()]
    format = getFirstNImageFormat ( jsonSubt_formats  )
    print(format[len(format)-1])

    interval_time_sort = centi_interval_times[i]
    
    writeImageFromTimes(jsonSubt_formats ,  interval_time_sort ,getTestDir(_id,dct_total,'img')  )
    
    writeMp3FromTimes(jsonSubt_formats , _id , interval_time_sort  , exec_ytdlp, getTestDir(_id,dct_total,'mp3')   )


In [ ]:
class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = "harshit345/xlsr-wav2vec-speech-emotion-recognition"
config = AutoConfig.from_pretrained(model_name_or_path)
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
sampling_rate = feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [ ]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech
def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    inputs = feature_extractor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    inputs = {key: inputs[key].to(device) for key in inputs}
    with torch.no_grad():
        logits = model(**inputs).logits
    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs

In [ ]:

sr = 16000
 
# if you change code below, it will download the dataset again 
db = audb.load(
    'emodb',
    version='1.1.1',
    format='wav',
    mixdown=True,
    sampling_rate=sr,
    full_path=False,
    verbose=True,
)
 
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.emobase,
    feature_level=opensmile.FeatureLevel.Functionals,
)

 

In [ ]:
@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [ ]:
path_2 = 'C:/Users/julien/audb/emodb/1.1.1/fe182b91/wav/10a07Wb.wav'   
outputs = predict(path_2, sampling_rate)

In [ ]:
print(outputs)
speech_array, _sampling_rate = torchaudio.load(path_2 )
print(speech_array.shape)

In [ ]:
lll= "D:/Master/S2/GestionProjet/test_video_.wav"
speech_array, _sampling_rate = torchaudio.load(lll)
print(speech_array.shape)



In [ ]:

fs, data = wavfile.read(lll)            # reading the file

wavfile.write("D:/Master/S2/GestionProjet/audio_section_channel_1.wav", fs, data[:, 0])   # saving first column which corresponds to channel 1
wavfile.write("D:/Master/S2/GestionProjet/audio_section_channel_2.wav", fs, data[:, 1])   # saving second column which corresponds to channel 2

In [ ]:

outputs_2 = predict("D:/Master/S2/GestionProjet/audio_section_channel_2.wav", sampling_rate)
print(outputs_2)

In [ ]:

lll="D:/Master/S2/GestionProjet/billy-scream.wav"
fs, data = wavfile.read(lll)            # reading the file

wavfile.write("D:/Master/S2/GestionProjet/billy-scream_channel_1.wav", fs, data[:, 0])   # saving first column which corresponds to channel 1
wavfile.write("D:/Master/S2/GestionProjet/billy-scream_channel_2.wav", fs, data[:, 1])   # saving second column which corresponds to channel 2

In [ ]:



classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")
out_prob, score, index, text_lab = classifier.classify_file("D:/Master/S2/GestionProjet/billy-scream_channel_1.wav")
print(text_lab)